In [ ]:
import pandas as pd

df = pd.read_table('list_landmarks_inshop.txt', sep = '\s+', skiprows = 1)

df

In [ ]:
n = 100

dfa = df[(df['clothes_type'] == 1) & (df['variation_type'] == 1) &
         (df['landmark_visibility_1'] == 0) & (df['landmark_visibility_2'] == 0) & 
         (df['landmark_visibility_3'] == 0) & (df['landmark_visibility_4'] == 0) &
         (df['landmark_visibility_5'] == 0) & (df['landmark_visibility_6'] == 0)][:n]

dfa

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([ img_to_array(load_img(f)) for f in dfa['image_name']])

imgs.shape

In [ ]:
cols = [f'landmark_location_{t}_{i + 1}' for i in range(6) for t in ['x', 'y'] ]
cols

In [ ]:
labels = dfa[cols].values

for d in labels:
    for i in range(0, len(cols), 2):
        d[i] /= imgs.shape[2]
        d[i + 1] /= imgs.shape[1]

labels.shape

In [ ]:
labels[0]

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, GRU, Permute, Bidirectional, TimeDistributed, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

input = Input(shape = imgs.shape[1:])

x = input

x = BatchNormalization()(x)

x = Conv2D(1, 3, padding='same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.4)(x)

sh = (int(x.shape[1]), int(x.shape[2]))

x = Reshape(sh)(x)

x = Bidirectional(GRU(64, return_sequences = True, activation = 'relu'))(x)
x = Dropout(0.4)(x)

x = Bidirectional(GRU(64, return_sequences = True, activation = 'relu'))(x)
x = Dropout(0.4)(x)

x = TimeDistributed(Dense(64, activation = 'relu'))(x)
x = Dropout(0.4)(x)

x = Flatten()(x)

x = Dense(512, activation = 'relu')(x)
x = Dropout(0.4)(x)

x = Dense(256, activation = 'relu')(x)
x = Dropout(0.4)(x)

output = Dense(labels.shape[-1], activation = 'sigmoid')(x)

model = Model(inputs = input, outputs = output)

model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])

model.summary()

In [ ]:
from keras.models import load_model

model = load_model('model/r1_0.h5')

In [ ]:
hist = model.fit(imgs, labels, initial_epoch = 0, epochs = 200, batch_size = 10, validation_split = 0.2)

hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (16., 4.)

plt.subplot(1, 4, 1)
plt.plot(hist.history['loss'])

plt.subplot(1, 4, 2)
plt.plot(hist.history['mean_absolute_error'])

plt.subplot(1, 4, 3)
plt.plot(hist.history['val_loss'])

plt.subplot(1, 4, 4)
plt.plot(hist.history['val_mean_absolute_error'])




In [ ]:
model.save('model/rnn1_0.h5')

In [ ]:
import cv2

colors = [(255, 255, 255), (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255), (255, 165, 0), (210, 180, 140)]

def predict(index, c_size = 6, s = 4.0):
    plt.rcParams['figure.figsize'] = (s * 2, s)
    
    img = imgs[index]

    p = model.predict(np.array([img]))[0]

    img1 = img.astype(int)
    img2 = img.astype(int)
    
    for i in range(0, len(p), 2):
        color = colors[int(i / 2) + 1]
        
        dx = int(labels[index][i] * imgs.shape[2])
        dy = int(labels[index][i + 1] * imgs.shape[1])

        img1 = cv2.circle(img1, (dx, dy), c_size, color, -1)
        
        px = int(p[i] * imgs.shape[2])
        py = int(p[i + 1] * imgs.shape[1])

        img2 = cv2.circle(img2, (px, py), c_size, color, -1)

        print(f"label: ({dx}, {dy}), predict: ({px}, {py})")

    plt.subplot(1, 2, 1)
    plt.imshow(img1)

    plt.subplot(1, 2, 2)
    plt.imshow(img2)


In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)

In [ ]:
predict(10)

In [ ]:
predict(15)

In [ ]:
predict(-1)

In [ ]:
predict(-2)

In [ ]:
predict(-3)

In [ ]:
predict(-4)